# Notebook: Entraînement NN sur Google Colab

Ce notebook montre comment préparer l'environnement Colab, monter Google Drive, installer les dépendances, charger les données du dépôt `smart-chess` et lancer l'entraînement via `ai/train_torch.py`.

Instructions rapides :
- Ouvrez Runtime -> Change runtime type -> GPU
- Montez Google Drive si vous voulez sauvegarder checkpoints persistants
- Placez votre dataset `chessData.csv` dans le dossier du projet (ou copiez-le depuis Drive)


## Avant de commencer — choisissez runtime GPU

Assurez-vous d'avoir sélectionné Runtime -> Change runtime type -> GPU dans Google Colab.

Ce bloc affiche la version de Python, la version de torch et si un GPU est disponible.

In [ ]:
# Vérification GPU et versions
import sys
import platform

print('Python:', sys.version)
print('Platform:', platform.platform())

try:
    import torch
    print('PyTorch:', torch.__version__)
    print('CUDA available:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('Device name:', torch.cuda.get_device_name(0))
        prop = torch.cuda.get_device_properties(0)
        print(f'  Total memory: {prop.total_memory / 1e9:.2f} GB')
except Exception as e:
    print('PyTorch non installé ou erreur:', e)


In [ ]:
# Installer / mettre à jour dépendances (exécuter seulement si nécessaire)
# En Colab, PyTorch est souvent préinstallé. Décommentez la ligne d'installation si besoin.

# %%bash
# pip install -q torch torchvision torchaudio

# Dépendances utiles
!pip install -q pandas tqdm numpy

print('Installation effectuée (ou déjà présente).')


## Monter Google Drive

Montez Google Drive pour sauvegarder checkpoints et datasets persistants. Exécutez cette cellule et suivez l'URL d'authentification.

Remarque : l'emplacement `MyDrive` est accessible sous `/content/drive/MyDrive`.


In [ ]:
# Code : monter Drive et créer dossier de checkpoints
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    ckpt_dir = '/content/drive/MyDrive/smart-chess-checkpoints'
    os.makedirs(ckpt_dir, exist_ok=True)
    print('Drive monté et dossier de checkpoints prêt:', ckpt_dir)
except Exception as e:
    print('Erreur (peut-être non exécuté sur Colab) :', e)
    print('Si vous n\'êtes pas sur Colab, créez un dossier local pour les checkpoints.')


## Obtenir le code du dépôt

Option A — Cloner depuis GitHub (rapide) :

- Avantage : récupère la version du dépôt telle qu'elle est sur GitHub.
- Si vous avez des modifications locales, préférez copier depuis Drive.


In [ ]:
# Cloner le repo GitHub (décommentez si besoin)
# !git clone https://github.com/promaaa/smart-chess.git /content/smart-chess
# %cd /content/smart-chess

# Ou, si vous avez uploadé le repo dans Drive, copiez depuis Drive:
# !cp -r /content/drive/MyDrive/path/to/smart-chess /content/smart-chess

print('Si le repo n\'est pas présent, décommentez et exécutez la commande git clone ci-dessus.')


## Configurer le répertoire de travail et vérifier les fichiers importants

Le code d'entraînement se trouve dans `ai/train_torch.py` et `ai/train.py`. Assurez-vous que le dossier `smart-chess` est dans `/content` ou montez le chemin depuis Drive.


In [ ]:
# Exemple : définir le répertoire de travail et ajouter le repo au PYTHONPATH
import os, sys

# Ajustez si vous avez cloné dans /content/smart-chess ou copié depuis Drive
repo_path = '/content/smart-chess'
if not os.path.exists(repo_path):
    # Si le repo est dans Drive, vous pouvez le pointer ici :
    repo_path = '/content/drive/MyDrive/smart-chess'  # ajustez selon votre arborescence Drive

print('Repo path:', repo_path)
if os.path.exists(repo_path):
    sys.path.insert(0, repo_path)
    os.chdir(repo_path)
    print('Contenu du repo (quelques fichiers):')
    print(os.listdir(repo_path)[:30])
else:
    print('Repo introuvable. Clonez-le ou copiez-le depuis Drive avant de continuer.')


## Préparer / vérifier le dataset

Le script d'entraînement attend un fichier `chessData.csv` (format FEN,Evaluation). Placez-le à la racine du repo ou dans Drive et mettez à jour `DATASET_PATH` avant d'exécuter l'entraînement.

Cette cellule cherche automatiquement le dataset dans plusieurs emplacements et le copie dans `/content/smart-chess` si nécessaire.

In [ ]:
# Chercher le dataset et le copier dans le repo si besoin
import os, shutil

def find_and_copy_dataset():
    possible = [
        '/content/smart-chess/chessData.csv',
        '/content/drive/MyDrive/chessData.csv',
        '/content/drive/MyDrive/smart-chess/chessData.csv'
    ]
    for p in possible:
        if os.path.exists(p):
            dest = '/content/smart-chess/chessData.csv'
            os.makedirs(os.path.dirname(dest), exist_ok=True)
            if os.path.abspath(p) != os.path.abspath(dest):
                shutil.copy(p, dest)
                print('Dataset copié:', p, '→', dest)
            else:
                print('Dataset trouvé:', dest)
            return dest
    print('Dataset non trouvé. Téléversez `chessData.csv` dans Drive ou placez-le dans le repo.')
    return None

DATASET_PATH = find_and_copy_dataset()
print('DATASET_PATH =', DATASET_PATH)


## Lancer l'entraînement (exemple)

Cette cellule montre comment appeler `ai/train_torch.py` depuis le notebook. Vous pouvez surcharger les constantes du module avant d'appeler `main()` pour rediriger les checkpoints vers Drive ou réduire `EPOCHS`/`MAX_SAMPLES` pour un test rapide.

Exécutez seulement après avoir copié le dataset et configuré `repo_path`.


In [ ]:
# Exemple d'appel à ai.train_torch
import importlib

try:
    import ai.train_torch as trainer
    # Rediriger les fichiers de sortie vers Drive pour persistance
    ckpt_dir = '/content/drive/MyDrive/smart-chess-checkpoints'
    if os.path.exists(ckpt_dir):
        trainer.WEIGHTS_FILE = os.path.join(ckpt_dir, 'chess_nn_weights.npz')
        trainer.CHECKPOINT_FILE = os.path.join(ckpt_dir, 'chess_model_checkpoint.pt')
    # Réduire pour test rapide
    trainer.EPOCHS = 2
    trainer.MAX_SAMPLES = 5000
    # Si DATASET_PATH trouvé plus haut, l'utiliser
    if DATASET_PATH:
        trainer.DATASET_PATH = DATASET_PATH

    print('Configuration trainer:')
    print(' DATASET_PATH=', trainer.DATASET_PATH)
    print(' WEIGHTS_FILE=', trainer.WEIGHTS_FILE)
    print(' CHECKPOINT_FILE=', trainer.CHECKPOINT_FILE)
    print(' EPOCHS=', trainer.EPOCHS)
    print(' MAX_SAMPLES=', trainer.MAX_SAMPLES)

    # Lancer l'entraînement (bloquant) — surveillez la sortie
    trainer.main()

except Exception as e:
    print('Erreur lors du lancement de l\'entraînement:', e)
    print('Vérifiez que le repo est dans le PYTHONPATH et que les dépendances sont installées.')


## Sauvegarder / copier les checkpoints manuellement vers Drive

Si le script a enregistré des fichiers localement, exécutez cette cellule pour les copier dans le dossier Drive `smart-chess-checkpoints`.


In [ ]:
# Copier manuellement les fichiers de checkpoint vers Drive
import os, shutil
ckpt_dir = '/content/drive/MyDrive/smart-chess-checkpoints'
files_to_copy = ['chess_model_checkpoint.pt', 'chess_nn_weights.npz']
for f in files_to_copy:
    if os.path.exists(f):
        dest = os.path.join(ckpt_dir, f)
        shutil.copy(f, dest)
        print('Copié:', f, '→', dest)
    else:
        print('Fichier absent (pas de sauvegarde locale trouvée) :', f)


## Conseils & debug rapide

- Pour un test rapide : réduire `trainer.EPOCHS` à 1-2 et `trainer.MAX_SAMPLES` à 1k-5k.
- Vérifiez la présence d'un GPU avec `torch.cuda.is_available()` ; ajustez `BATCH_SIZE` si OOM.
- Si l'entraînement est lent : augmentez `BATCH_SIZE` et activez `pin_memory=True` (déjà géré dans `train_torch.py`).
- Sauvegardes : les fichiers `chess_model_checkpoint.pt` et `chess_nn_weights.npz` sont créés automatiquement si la configuration par défaut est utilisée.
- Logs : surveillez la sortie de la cellule d'entraînement pour RMSE / perte et sauvegarde de checkpoint.

Si vous voulez, je peux :
- Ajuster le notebook pour utiliser `torch.cuda.amp` (mixed precision) pour accélérer l'entraînement sur GPU modernes.
- Ajouter une cellule pour visualiser les courbes de loss/metrics après l'entraînement.
